In [1]:
#------------------------------------#
#------------- packages--------------#
#------------------------------------#

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from datetime import datetime, timedelta, date

import requests

import os

In [2]:
pids = pd.read_csv('StudyParticipants.csv')
aware_ids = pd.read_csv('aware_device.csv')

# merge the two dataframes on ParticipantID and keep only the rows from pids
pids = pd.merge(pids, aware_ids, on='ParticipantID', how='left')
pids

,ParticipantID,Phone,Awair,AwairID,Work,device_id,brand,build_id,manufacturer,model
0,edr,iPhone,AW-13145,70886b1232f6,Link Lab,393b2587-70fc-4e41-9368-7a14648375e3,iPhone,Darwin Kernel Version 21.4.0: Mon Feb 21 21:27...,Apple,iPhone
1,egl,iPhone,AW-13145,70886b1232f6,Link Lab,a268bf35-24a7-40eb-a1fc-94142ffd3e58,iPhone,Darwin Kernel Version 21.4.0: Mon Feb 21 21:27...,Apple,iPhone
2,j02,iPhone,AW-25396,70886b125db1,Off Grounds BR,9e61c8ae-5a4f-41b0-80ba-08ab06d495c6,iPhone,Darwin Kernel Version 21.3.0: Wed Jan 5 21:44...,Apple,iPhone
3,uja,iPhone,AW-16515,70886b123935,Link Lab,0dd46ec9-e40a-4791-9cee-3058dadc1959,iPhone,Darwin Kernel Version 21.4.0: Mon Feb 21 21:27...,Apple,iPhone
4,lpz,Android,AW-29799,70886b127d5c,Off Grounds BR,5f863ec2-9314-41eb-8425-bf7173194f00,samsung,SP1A.210812.016.A526U1UES5CVD1,Samsung,SM-A526U1
5,oyb,Android,AW-23619,70886b126976,Link Lab,18519687-7d50-4493-8bdb-309d4d29dfd8,google,SP2A.220305.012,Google,Pixel 4a
6,mr1,iPhone,AW-23810,70886b126a7b,Link Lab,78004874-9c43-4d4f-8962-2db67c6ffe4a,iPhone,Darwin Kernel Version 21.4.0: Mon Feb 21 21:27...,Apple,iPhone
7,xil,Android,AW-22996,70886b12532b,Link Lab,08d621fd-54ca-43de-9102-46bb2a17cce6,google,SP2A.220505.002,Google,Pixel 4a
8,pgm,iPhone,AW-15886,70886b123bc8,Link Lab,f7a16ee5-b15c-4034-ba52-4becf44cc74b,iPhone,Darwin Kernel Version 21.4.0: Mon Feb 21 21:27...,Apple,iPhone
9,h9u,iPhone,AW-27185,70886b12697a,Off Grounds,04346571-51a9-4272-b6b1-52ac48248515,iPhone,Darwin Kernel Version 21.3.0: Wed Jan 5 21:44...,Apple,iPhone


In [3]:
aware = []

for r,d,f in os.walk("/Users/beatriceli/Documents/PhD_Research/GitHub/well-being/lll_aware"):
    for file in f:
        if file.endswith("screen.csv"):
            temp = pd.read_csv(os.path.join(r,file),parse_dates=["device_id"])
            aware.append(temp)

aware = pd.concat(aware)
# create columns called "ParticipantID" and "Work" and get value from matching device_id in aware to device_id in pids
# if device_ide in aware matches device_id in pids, then get the value of ParticipantID and Work from pids
aware = pd.merge(aware, pids[['device_id', 'ParticipantID', 'Work']], on='device_id', how='left')
# rename device_id.1 to screen_status
aware = aware.rename(columns={'device_id.1': 'screen_status'})
# convert epoch time to datetime and round to nearest second
aware['timestamp'] = pd.to_datetime(aware['timestamp'], unit='ms').dt.round('s')

aware.head()

,_id,timestamp,device_id,screen_status,ParticipantID,Work
0,13111,2022-05-06 14:29:41,3c23fa1b-4058-4685-bb71-ca06ffea3757,2,vxx,Link Lab
1,13112,2022-05-06 14:35:09,3c23fa1b-4058-4685-bb71-ca06ffea3757,3,vxx,Link Lab
2,13113,2022-05-06 14:36:13,3c23fa1b-4058-4685-bb71-ca06ffea3757,2,vxx,Link Lab
3,13114,2022-05-06 14:41:17,3c23fa1b-4058-4685-bb71-ca06ffea3757,3,vxx,Link Lab
4,13115,2022-05-06 14:42:50,3c23fa1b-4058-4685-bb71-ca06ffea3757,2,vxx,Link Lab


In [4]:
# # get number of rows with no ParticipantID
# aware['ParticipantID'].isnull().sum()
# # get device_id for rows with no ParticipantID
# aware[aware['ParticipantID'].isnull()]['device_id'].unique() # only one device-id with no ParticipantID: '42f346aa-b3ea-435a-b912-c5dec85b7f8b'
# remove unecessary columns
# drop _id, device_id
aware = aware.drop(['_id', 'device_id'], axis=1)
# create date column
aware['date'] = aware['timestamp'].dt.date
# move ParticipantID and Work to the front of the dataframe
aware = aware[['ParticipantID', 'Work', 'date','timestamp', 'screen_status']]
# remove rows with no ParticipantID
aware = aware[aware['ParticipantID'].notna()]
# sort by ParticipantID and timestamp
aware = aware.sort_values(by=['ParticipantID', 'timestamp'])
aware.head()

,ParticipantID,Work,date,timestamp,screen_status
6595,edr,Link Lab,2022-04-26,2022-04-26 14:53:49,2
6596,edr,Link Lab,2022-04-26,2022-04-26 14:58:00,3
6597,edr,Link Lab,2022-04-26,2022-04-26 15:01:29,2
6598,edr,Link Lab,2022-04-26,2022-04-26 15:01:31,3
6599,edr,Link Lab,2022-04-26,2022-04-26 15:01:43,2


In [5]:
aware.shape[0]

141554

In [6]:
# get first and last timestamp for each ParticipantID
aware.groupby('ParticipantID')['timestamp'].agg(['min', 'max'])

,min,max
ParticipantID,,
edr,2022-04-26 14:53:49,2022-07-22 20:21:14
egl,2022-04-26 14:57:05,2022-07-04 03:46:03
h9u,2022-05-05 16:29:33,2022-07-24 23:13:56
heh,2022-04-28 17:00:17,2022-08-10 16:41:03
j02,2022-04-28 15:50:14,2022-06-22 16:46:41
lpz,2022-04-28 16:52:38,2022-07-13 00:24:11
mr1,2022-05-04 14:41:52,2022-07-08 16:52:04
oyb,2022-04-28 16:42:25,2022-04-29 14:59:41
pgm,2022-05-05 15:59:29,2022-06-28 15:35:41


In [7]:
# create time_start and time_end columns
aware['time_start'] = aware['timestamp']
aware['time_end'] = aware['timestamp'].shift(-1)

# group by ParticipantID and Work, and forward fill time_end values
aware['time_end'] = aware.groupby(['ParticipantID', 'Work'])['time_end'].fillna(method='ffill')

# for each ParticipantID, drop the last row
aware = aware.groupby('ParticipantID').apply(lambda x: x.iloc[:-1]).reset_index(drop=True)
aware.head()

,ParticipantID,Work,date,timestamp,screen_status,time_start,time_end
0,edr,Link Lab,2022-04-26,2022-04-26 14:53:49,2,2022-04-26 14:53:49,2022-04-26 14:58:00
1,edr,Link Lab,2022-04-26,2022-04-26 14:58:00,3,2022-04-26 14:58:00,2022-04-26 15:01:29
2,edr,Link Lab,2022-04-26,2022-04-26 15:01:29,2,2022-04-26 15:01:29,2022-04-26 15:01:31
3,edr,Link Lab,2022-04-26,2022-04-26 15:01:31,3,2022-04-26 15:01:31,2022-04-26 15:01:43
4,edr,Link Lab,2022-04-26,2022-04-26 15:01:43,2,2022-04-26 15:01:43,2022-04-26 15:02:24


In [8]:
# remove timestamp
aware = aware.drop('timestamp', axis=1)
# create duration column in seconds
aware['duration'] = (aware['time_end'] - aware['time_start']).dt.total_seconds()

In [9]:
# what participant has a screen_status of 0 and 1?
aware[aware['screen_status'] == 0]['ParticipantID'].unique()
aware[aware['screen_status'] == 1]['ParticipantID'].unique()

# from pids, make list called apple for participants with iphones
applepids = pids[pids['model'] == 'iPhone']['ParticipantID'].tolist()
# from pids, make list called android for participants with non-iphones
androidpids = pids[pids['model'] != 'iPhone']['ParticipantID'].tolist()

In [10]:
# subset aware into apple and android
apple = aware[aware['ParticipantID'].isin(applepids)]
android = aware[aware['ParticipantID'].isin(androidpids)]

# unique values in screen_status for apple
apple['screen_status'].unique()

array([2, 3])

In [11]:
# iphones only have 2 and 3 as screen_status
# map 2=locked and 3=unlocked
apple['screen_status'] = apple['screen_status'].map({2: 'locked', 3: 'unlocked'})
apple.head()

/var/folders/zd/_7gwj5650vb4smtfpkjw268h0000gn/T/ipykernel_8696/650208130.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apple['screen_status'] = apple['screen_status'].map({2: 'locked', 3: 'unlocked'})


,ParticipantID,Work,date,screen_status,time_start,time_end,duration
0,edr,Link Lab,2022-04-26,locked,2022-04-26 14:53:49,2022-04-26 14:58:00,251.0
1,edr,Link Lab,2022-04-26,unlocked,2022-04-26 14:58:00,2022-04-26 15:01:29,209.0
2,edr,Link Lab,2022-04-26,locked,2022-04-26 15:01:29,2022-04-26 15:01:31,2.0
3,edr,Link Lab,2022-04-26,unlocked,2022-04-26 15:01:31,2022-04-26 15:01:43,12.0
4,edr,Link Lab,2022-04-26,locked,2022-04-26 15:01:43,2022-04-26 15:02:24,41.0


In [12]:
# are there duration values greater than 80000 seconds?
apple[apple['duration'] > 80000]

,ParticipantID,Work,date,screen_status,time_start,time_end,duration
11375,edr,Link Lab,2022-07-14,unlocked,2022-07-14 18:34:32,2022-07-15 17:25:06,82234.0
22809,egl,Link Lab,2022-07-02,unlocked,2022-07-02 05:03:09,2022-07-03 19:33:46,138637.0
70984,j02,Off Grounds BR,2022-06-15,locked,2022-06-15 03:00:17,2022-06-16 19:28:18,145681.0
70985,j02,Off Grounds BR,2022-06-16,locked,2022-06-16 19:28:18,2022-06-22 16:40:27,508329.0
105270,ssg,Link Lab,2022-05-25,locked,2022-05-25 01:43:06,2022-05-28 17:26:37,315811.0
125789,vxx,Link Lab,2022-06-02,unlocked,2022-06-02 18:58:54,2022-06-04 19:17:52,173938.0


In [13]:
# to csv
apple.to_csv('apple_screen.csv', index=False)

In [14]:
def split_rows_by_day(df):
    """
    Splits rows in a pandas DataFrame by day when time_start and time_end span across multiple days.
    
    Args:
        df (pd.DataFrame): Input DataFrame with columns 'time_start', 'time_end', and 'time_spent' in datetime format.
        
    Returns:
        pd.DataFrame: DataFrame with rows split by day when time_start and time_end span across multiple days.
    """
    # Create a copy of the input DataFrame to avoid modifying the original data
    df_split = df.copy()
    
    # Identify rows where time_start and time_end span across multiple days
    multi_day_rows = df_split[df_split['time_start'].dt.date != df_split['time_end'].dt.date]
    
    # Create a list to store the new rows
    new_rows = []
    
    # Iterate over multi-day rows
    for idx, row in multi_day_rows.iterrows():
        # Extract the date portion from the datetime objects
        date_start = row['time_start'].date()
        date_end = row['time_end'].date()

        # Calculate the start and end times for the first day
        end_time_day1 = datetime.combine(date_start, datetime.max.time())  # End of the day (23:59:59)
        start_time_day2 = datetime.combine(date_end, datetime.min.time())  # Beginning of the day (00:00:00) of the subsequent day

        # Calculate the time spent for the first day and the subsequent day
        time_spent_day1 = (end_time_day1 - row['time_start']).total_seconds()
        time_spent_day2 = (row['time_end'] - start_time_day2).total_seconds()

        # Round the time spent for the first day and the subsequent day to the nearest second
        time_spent_day1 = round(time_spent_day1)
        time_spent_day2 = round(time_spent_day2)

        # Update the original row for the first day
        df_split.at[idx, 'time_end'] = end_time_day1
        df_split.at[idx, 'duration'] = time_spent_day1

        # Create a new row for the subsequent day
        new_row = row.copy()
        new_row['time_start'] = start_time_day2
        new_row['time_end'] = row['time_end']
        new_row['duration'] = time_spent_day2
        new_row['date'] = date_end

        # Append the new row to the list
        new_rows.append(new_row)

    # Convert the list of dictionaries into a DataFrame
    new_rows_df = pd.DataFrame(new_rows)
    
    # Concatenate the new rows with the original DataFrame
    df_split = pd.concat([df_split, new_rows_df], ignore_index=True)
    
    # Sort the DataFrame by ParticipantID and time_start
    df_split = df_split.sort_values(['ParticipantID', 'time_start']).reset_index(drop=True)
    # Round down the time_end to the nearest second to avoid rounding up to the next day
    df_split['time_end'] = df_split['time_end'].dt.floor('1s')

    return df_split

In [15]:
apple = split_rows_by_day(apple)
apple[apple['time_start'].dt.date != apple['time_end'].dt.date]

,ParticipantID,Work,date,screen_status,time_start,time_end,duration


In [16]:
# max duration for each ParticipantID
apple.groupby('ParticipantID')['duration'].max()

ParticipantID
edr    62706.0
egl    70426.0
h9u    75847.0
j02    75583.0
mr1    36405.0
pgm    46470.0
ssg    80214.0
uja    29368.0
vxx    69472.0
Name: duration, dtype: float64

In [17]:
# groupby ParticipantID, Work and date
# sum duration for each screen_status
# reset index
applesum = apple.groupby(['ParticipantID', 'Work', 'date', 'screen_status'])['duration'].sum().reset_index()
applesum.head()

,ParticipantID,Work,date,screen_status,duration
0,edr,Link Lab,2022-04-26,locked,26561.0
1,edr,Link Lab,2022-04-26,unlocked,6210.0
2,edr,Link Lab,2022-04-27,locked,48574.0
3,edr,Link Lab,2022-04-27,unlocked,37826.0
4,edr,Link Lab,2022-04-28,locked,69714.0


In [18]:
# get only unlocked in applesum and name it appleuse
appleuse = applesum[applesum['screen_status'] == 'unlocked']
# drop screen_status column
appleuse = appleuse.drop('screen_status', axis=1)
# rename duration to unlocked_duration
appleuse = appleuse.rename(columns={'duration': 'unlocked_duration'})
appleuse.head()

,ParticipantID,Work,date,unlocked_duration
1,edr,Link Lab,2022-04-26,6210.0
3,edr,Link Lab,2022-04-27,37826.0
5,edr,Link Lab,2022-04-28,16686.0
7,edr,Link Lab,2022-04-29,12248.0
9,edr,Link Lab,2022-04-30,23136.0


In [19]:
# 0=off, 1=on, 2=locked, 3=unlocked
android['screen_status'] = android['screen_status'].map({0: 'off', 1: 'on', 2: 'locked', 3: 'unlocked'})
android.head()

/var/folders/zd/_7gwj5650vb4smtfpkjw268h0000gn/T/ipykernel_8696/3851141433.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  android['screen_status'] = android['screen_status'].map({0: 'off', 1: 'on', 2: 'locked', 3: 'unlocked'})


,ParticipantID,Work,date,screen_status,time_start,time_end,duration
35489,heh,Off Grounds,2022-04-28,on,2022-04-28 17:00:17,2022-04-28 17:00:17,0.0
35490,heh,Off Grounds,2022-04-28,unlocked,2022-04-28 17:00:17,2022-04-28 17:00:18,1.0
35491,heh,Off Grounds,2022-04-28,on,2022-04-28 17:00:18,2022-04-28 17:00:18,0.0
35492,heh,Off Grounds,2022-04-28,unlocked,2022-04-28 17:00:18,2022-04-28 17:03:49,211.0
35493,heh,Off Grounds,2022-04-28,off,2022-04-28 17:03:49,2022-04-28 17:03:49,0.0


In [20]:
# for android
android = split_rows_by_day(android)
android[android['time_start'].dt.date != android['time_end'].dt.date]

,ParticipantID,Work,date,screen_status,time_start,time_end,duration


In [21]:
andsum = android.groupby(['ParticipantID', 'Work', 'date', 'screen_status'])['duration'].sum().reset_index()
andsum.head()

,ParticipantID,Work,date,screen_status,duration
0,heh,Off Grounds,2022-04-28,locked,14537.0
1,heh,Off Grounds,2022-04-28,off,385.0
2,heh,Off Grounds,2022-04-28,on,2919.0
3,heh,Off Grounds,2022-04-28,unlocked,7342.0
4,heh,Off Grounds,2022-04-29,locked,55682.0


In [22]:
# get only unlocked and on in andsum and name it anduse
anduse = andsum[andsum['screen_status'].isin(['unlocked'])]
anduse.head()

,ParticipantID,Work,date,screen_status,duration
3,heh,Off Grounds,2022-04-28,unlocked,7342.0
7,heh,Off Grounds,2022-04-29,unlocked,30529.0
11,heh,Off Grounds,2022-04-30,unlocked,28778.0
15,heh,Off Grounds,2022-05-01,unlocked,80547.0
17,heh,Off Grounds,2022-05-04,unlocked,86400.0


In [23]:
# add up duration for unlocked and on
anduse = anduse.groupby(['ParticipantID', 'Work', 'date'])['duration'].sum().reset_index()
# rename duration to unlocked_duration
anduse = anduse.rename(columns={'duration': 'unlocked_duration'})
anduse.head()

,ParticipantID,Work,date,unlocked_duration
0,heh,Off Grounds,2022-04-28,7342.0
1,heh,Off Grounds,2022-04-29,30529.0
2,heh,Off Grounds,2022-04-30,28778.0
3,heh,Off Grounds,2022-05-01,80547.0
4,heh,Off Grounds,2022-05-04,86400.0


In [24]:
# get mean unlocked_duration for each participant
appleuse.groupby('ParticipantID')['unlocked_duration'].mean()

ParticipantID
edr    18859.238636
egl    14661.000000
h9u    12720.259259
j02    17328.960000
mr1    10148.393939
pgm    10369.818182
ssg    16271.939759
uja    19528.477612
vxx    13079.763636
Name: unlocked_duration, dtype: float64

In [25]:
anduse.groupby('ParticipantID')['unlocked_duration'].mean()

ParticipantID
heh    45852.051724
lpz    20050.960784
oyb     3238.500000
xil    25407.290323
Name: unlocked_duration, dtype: float64

In [26]:
apple.head()

,ParticipantID,Work,date,screen_status,time_start,time_end,duration
0,edr,Link Lab,2022-04-26,locked,2022-04-26 14:53:49,2022-04-26 14:58:00,251.0
1,edr,Link Lab,2022-04-26,unlocked,2022-04-26 14:58:00,2022-04-26 15:01:29,209.0
2,edr,Link Lab,2022-04-26,locked,2022-04-26 15:01:29,2022-04-26 15:01:31,2.0
3,edr,Link Lab,2022-04-26,unlocked,2022-04-26 15:01:31,2022-04-26 15:01:43,12.0
4,edr,Link Lab,2022-04-26,locked,2022-04-26 15:01:43,2022-04-26 15:02:24,41.0


In [27]:
# group by ParticipantID, Work, date
# get count of different screen_status

# apple
applecount = apple.groupby(['ParticipantID', 'Work', 'date']).agg(
    {'screen_status': ['count', 'nunique']},
    {'duration':'sum'}).reset_index()
applecount.head()

ParticipantID      Work        date screen_status        
                                              count nunique
0           edr  Link Lab  2022-04-26            76       2
1           edr  Link Lab  2022-04-27           123       2
2           edr  Link Lab  2022-04-28            97       2
3           edr  Link Lab  2022-04-29           101       2
4           edr  Link Lab  2022-04-30           197       2

In [28]:
# create a pivot table for apple 
# index=ParticipantID, Work, date
# want the count and duration of each screen_status

table = apple.pivot_table(index=['ParticipantID', 'Work', 'date'], columns='screen_status', values='duration', aggfunc=['count', 'sum'])
# flatten the pivot table
table.columns = ['_'.join(col) for col in table.columns]
table = table.reset_index()
# to dataframe
table = pd.DataFrame(table)
table

,ParticipantID,Work,date,count_locked,count_unlocked,sum_locked,sum_unlocked
0,edr,Link Lab,2022-04-26,38.0,38.0,26561.0,6210.0
1,edr,Link Lab,2022-04-27,61.0,62.0,48574.0,37826.0
2,edr,Link Lab,2022-04-28,49.0,48.0,69714.0,16686.0
3,edr,Link Lab,2022-04-29,51.0,50.0,74152.0,12248.0
4,edr,Link Lab,2022-04-30,98.0,99.0,63264.0,23136.0
...,...,...,...,...,...,...,...
611,vxx,Link Lab,2022-06-26,32.0,31.0,68680.0,17720.0
612,vxx,Link Lab,2022-06-27,102.0,101.0,73580.0,12820.0
613,vxx,Link Lab,2022-06-28,73.0,72.0,64973.0,21427.0
614,vxx,Link Lab,2022-06-29,84.0,83.0,66876.0,19524.0


In [29]:
# to csv
table.to_csv('screen_status.csv', index=False)

# Screen Time before bed?

In [30]:
# read in sleep and survey data
fitsurveys = pd.read_csv('slpsteps.csv',parse_dates=['date'])
# cast to datetime
fitsurveys['bedtime'] = pd.to_datetime(fitsurveys['bedtime'])
fitsurveys['waketime'] = pd.to_datetime(fitsurveys['wakeup'])
# rename pid to ParticipantID
fitsurveys.rename(columns={'pid':'ParticipantID'},inplace=True)

In [31]:
fitsurveys.head()

,ParticipantID,group,wakeDate,day_name,bedtime,wakeup,duration,TST,REM:nREM,deep,...,Q27_1,Q34,Q34_6_TEXT,Q35,date,steps,steps_norm,steps_std,week,waketime
0,edr,1,2022-04-28,Thursday,2022-04-28 01:51:00,2022-04-28 10:30:00,31140.0,7.633333,0.275766,6060,...,5.0,"Neutral,Stressed",NaN,Work,2022-04-27,14713.0,0.685709,1.226819,weekday,2022-04-28 10:30:00
1,edr,2,2022-04-29,Friday,2022-04-29 01:13:00,2022-04-29 09:07:30,28470.0,7.475000,0.321060,4110,...,6.0,"Neutral,Happiness",NaN,"Work,Personal",2022-04-28,4316.0,0.075270,-1.145889,weekday,2022-04-29 09:07:30
2,edr,5,2022-05-02,Monday,2022-05-02 00:59:00,2022-05-02 08:27:00,26880.0,6.916667,0.328000,5760,...,6.0,Neutral,NaN,Work,2022-05-01,3034.0,0.000000,-1.438455,weekday,2022-05-02 08:27:00
3,edr,6,2022-05-03,Tuesday,2022-05-03 01:13:00,2022-05-03 08:22:30,25770.0,6.250000,0.315789,2100,...,6.0,Neutral,NaN,Work,2022-05-02,13436.0,0.610733,0.935394,weekday,2022-05-03 08:22:30
4,edr,7,2022-05-04,Wednesday,2022-05-04 01:24:30,2022-05-04 08:37:30,25980.0,6.683333,0.442446,3750,...,3.0,"Neutral,Stressed",NaN,Work,2022-05-03,14334.0,0.663457,1.140327,weekday,2022-05-04 08:37:30


In [32]:
# create new dataframe
# get ParticipantID, group, bedtime and wakeup
sleep = fitsurveys[['ParticipantID', 'group', 'bedtime', 'wakeup']]
# cast bedtime and wakeup to datetime
sleep['bedtime'] = pd.to_datetime(sleep['bedtime'])
sleep['wakeup'] = pd.to_datetime(sleep['wakeup'])
# create a column called date
sleep['b_date'] = sleep['bedtime'].dt.date
sleep['w_date'] = sleep['wakeup'].dt.date
# create a column called before_bed and after_wake to be one hour before bedtime and one hour after wakeup
sleep['before_bed'] = sleep['bedtime'] - pd.Timedelta(hours=1)
sleep['after_wake'] = sleep['wakeup'] + pd.Timedelta(hours=1)
sleep.head()

/var/folders/zd/_7gwj5650vb4smtfpkjw268h0000gn/T/ipykernel_8696/1022089231.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep['bedtime'] = pd.to_datetime(sleep['bedtime'])
/var/folders/zd/_7gwj5650vb4smtfpkjw268h0000gn/T/ipykernel_8696/1022089231.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep['wakeup'] = pd.to_datetime(sleep['wakeup'])
/var/folders/zd/_7gwj5650vb4smtfpkjw268h0000gn/T/ipykernel_8696/1022089231.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

,ParticipantID,group,bedtime,wakeup,b_date,w_date,before_bed,after_wake
0,edr,1,2022-04-28 01:51:00,2022-04-28 10:30:00,2022-04-28,2022-04-28,2022-04-28 00:51:00,2022-04-28 11:30:00
1,edr,2,2022-04-29 01:13:00,2022-04-29 09:07:30,2022-04-29,2022-04-29,2022-04-29 00:13:00,2022-04-29 10:07:30
2,edr,5,2022-05-02 00:59:00,2022-05-02 08:27:00,2022-05-02,2022-05-02,2022-05-01 23:59:00,2022-05-02 09:27:00
3,edr,6,2022-05-03 01:13:00,2022-05-03 08:22:30,2022-05-03,2022-05-03,2022-05-03 00:13:00,2022-05-03 09:22:30
4,edr,7,2022-05-04 01:24:30,2022-05-04 08:37:30,2022-05-04,2022-05-04,2022-05-04 00:24:30,2022-05-04 09:37:30


In [33]:
unlocked = pd.concat([apple, android], ignore_index=True)
# keep only unlocked
unlocked = unlocked[unlocked['screen_status'] == 'unlocked']
# rename time_start adn time_end to unlock_start and unlock_end
unlocked = unlocked.rename(columns={'time_start': 'unlock_start', 'time_end': 'unlock_end'})
# remove screen_status column
unlocked = unlocked.drop('screen_status', axis=1)
unlocked.head()

,ParticipantID,Work,date,unlock_start,unlock_end,duration
1,edr,Link Lab,2022-04-26,2022-04-26 14:58:00,2022-04-26 15:01:29,209.0
3,edr,Link Lab,2022-04-26,2022-04-26 15:01:31,2022-04-26 15:01:43,12.0
5,edr,Link Lab,2022-04-26,2022-04-26 15:02:24,2022-04-26 15:02:34,10.0
7,edr,Link Lab,2022-04-26,2022-04-26 15:07:09,2022-04-26 15:08:48,99.0
9,edr,Link Lab,2022-04-26,2022-04-26 15:25:27,2022-04-26 15:28:45,198.0


In [34]:
# if before_bed to bedtime is in range of time_start to time_end, then it is before_bed, flag screen_before as True
# if after_wake to wakeup is in range of time_start to time_end, then it is after_wake, flag screen_after as True
# with respect ParticipantID

# create a new column called screen_before and screen_after
sleep['screen_before'] = False
sleep['screen_after'] = False
sleep['screen_during'] = False
sleep['screen_duration'] = 0
sleep['screen_count'] = 0

# for each row in sleep
for index, row in sleep.iterrows():
    # get the ParticipantID for this row
    participant_id = row['ParticipantID']
    
    # get the before_bedtime for this row
    before_bedtime = row['before_bed']
    bedtime = row['bedtime']
    wakeup = row['wakeup']

    # get the unlocked rows for this participant
    unlocked_rows = unlocked[unlocked['ParticipantID'] == participant_id]
    
    # check if any unlocked rows are within one hour of before_bedtime
    for _, unlocked_row in unlocked_rows.iterrows():
        time_start = unlocked_row['unlock_start']
        time_end = unlocked_row['unlock_end']
        duration = unlocked_row['duration']
        
        # check if the unlocked time is within one hour of before_bedtime
        if (time_start <= before_bedtime <= time_end) or (time_start <= bedtime - pd.Timedelta(duration, unit='s') <= time_end):
            # screen use is within an hour of before_bedtime
            # set screen_before to True
            sleep.at[index, 'screen_before'] = True
            
        # check if the unlocked time is within one hour of wakeup
        if (time_start <= wakeup - pd.Timedelta(duration, unit='s') <= time_end) or (time_start <= wakeup <= time_end):
            # screen use is within an hour of wakeup
            # set screen_after to True
            sleep.at[index, 'screen_after'] = True

        # check if the unlocked time is between before_bedtime and wakeup
        if (bedtime <= time_start <= wakeup) or (bedtime <= time_end <= wakeup):
            # screen use is between before_bedtime and wakeup
            # set screen_during to True
            sleep.at[index, 'screen_during'] = True
            break

    # check if any unlocked rows are between bedtime and wakeup
    screen_during = unlocked_rows[(unlocked_rows['unlock_start'] >= bedtime) & (unlocked_rows['unlock_end'] <= wakeup)]['duration']
    # set screen_duration as the sum of durations within the range
    sleep.at[index, 'screen_duration'] = screen_during.sum()
    # set screen_count as the count of rows within the range
    sleep.at[index, 'screen_count'] = screen_during.count()

/var/folders/zd/_7gwj5650vb4smtfpkjw268h0000gn/T/ipykernel_8696/993777345.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep['screen_before'] = False
/var/folders/zd/_7gwj5650vb4smtfpkjw268h0000gn/T/ipykernel_8696/993777345.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sleep['screen_after'] = False
/var/folders/zd/_7gwj5650vb4smtfpkjw268h0000gn/T/ipykernel_8696/993777345.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [35]:
sleep

,ParticipantID,group,bedtime,wakeup,b_date,w_date,before_bed,after_wake,screen_before,screen_after,screen_during,screen_duration,screen_count
0,edr,1,2022-04-28 01:51:00,2022-04-28 10:30:00,2022-04-28,2022-04-28,2022-04-28 00:51:00,2022-04-28 11:30:00,False,False,True,11269,4
1,edr,2,2022-04-29 01:13:00,2022-04-29 09:07:30,2022-04-29,2022-04-29,2022-04-29 00:13:00,2022-04-29 10:07:30,False,False,True,2108,9
2,edr,5,2022-05-02 00:59:00,2022-05-02 08:27:00,2022-05-02,2022-05-02,2022-05-01 23:59:00,2022-05-02 09:27:00,False,False,True,4929,12
3,edr,6,2022-05-03 01:13:00,2022-05-03 08:22:30,2022-05-03,2022-05-03,2022-05-03 00:13:00,2022-05-03 09:22:30,False,False,True,6683,18
4,edr,7,2022-05-04 01:24:30,2022-05-04 08:37:30,2022-05-04,2022-05-04,2022-05-04 00:24:30,2022-05-04 09:37:30,False,False,True,6912,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...
363,xil,51,2022-06-23 00:14:30,2022-06-23 06:55:00,2022-06-23,2022-06-23,2022-06-22 23:14:30,2022-06-23 07:55:00,True,False,False,0,0
364,xil,52,2022-06-23 23:56:00,2022-06-24 07:36:00,2022-06-23,2022-06-24,2022-06-23 22:56:00,2022-06-24 08:36:00,False,False,False,0,0
365,xil,53,2022-06-24 22:57:30,2022-06-25 07:03:30,2022-06-24,2022-06-25,2022-06-24 21:57:30,2022-06-25 08:03:30,False,False,False,0,0
366,xil,54,2022-06-26 00:27:00,2022-06-26 06:22:30,2022-06-26,2022-06-26,2022-06-25 23:27:00,2022-06-26 07:22:30,False,False,False,0,0


In [36]:
# group by ParticipantID, get count of group, count of screen_before, count of screen_after
sleep.groupby(['ParticipantID']).agg({'group':'count','screen_before': 'sum', 'screen_after': 'sum', 'screen_during': 'sum', 'screen_duration':'mean', 'screen_count':'mean'}).reset_index()

,ParticipantID,group,screen_before,screen_after,screen_during,screen_duration,screen_count
0,edr,27,11,0,27,4470.962963,11.777778
1,egl,44,13,0,43,3944.068182,12.840909
2,h9u,58,18,0,57,2365.862069,13.896552
3,heh,53,30,17,39,3434.528302,8.641509
4,lpz,48,14,4,28,1927.187500,6.604167
5,mr1,28,10,0,28,3495.821429,18.964286
6,oyb,19,0,0,0,0.000000,0.000000
7,ssg,26,7,0,23,4893.153846,19.384615
8,uja,33,17,0,33,5610.484848,11.454545
9,xil,32,6,3,9,464.937500,2.906250
